# Dask jobqueue example

## What is Dask jobqueue? (<https://jobqueue.dask.org/>)

* deploys Dask workers on typical HPC job queueing systems

## Monte-Carlo estimate with multiple Dask batch job workers

We define a Dask jobqueue cluster with Dask workers that each have 8 CPUs and 48 GB of memory.

In [2]:
import dask, dask.distributed
import dask_jobqueue

In [3]:
#NOTE: Use command: ip link show
#      to confirm the name of the interconnect network (Infiniband)

cluster = dask_jobqueue.SLURMCluster(

    # Dask worker size
    cores=8, memory='10GB',
    processes=1, # Dask workers per job
    
    # SLURM job script things
    queue='CPU', walltime='00:20:00',
    
    # Dask worker network and temporary storage
    interface='ib0', local_directory= '/tmp' #'$TMPDIR'
)

client = dask.distributed.Client(cluster)

In [4]:
cluster.scale(jobs=2)

In [5]:
client

<Client: 'tcp://10.102.0.62:41808' processes=2 threads=16, memory=18.62 GiB>

### What is a jobqueue cluster?
The above is all we need to specify to run the computation on compute node Dask workers. 
Let's have a look at what's happening in the background.

In [6]:
!squeue -u $USER

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
             96175       CPU dask-wor  valerio  R       0:49      1 somacpu059
             96176       CPU dask-wor  valerio  R       0:49      1 somacpu060


In [7]:
print(cluster.job_script())

#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p CPU
#SBATCH -n 1
#SBATCH --cpus-per-task=8
#SBATCH --mem=10G
#SBATCH -t 00:20:00

/gpfs/soma_fs/home/valerio/anaconda3/envs/neuron/bin/python -m distributed.cli.dask_worker tcp://10.102.0.62:41808 --nthreads 8 --memory-limit 9.31GiB --name dummy-name --nanny --death-timeout 60 --local-directory /tmp --interface ib0 --protocol tcp://



In [8]:
client

<Client: 'tcp://10.102.0.62:41808' processes=2 threads=16, memory=18.62 GiB>

### Let's scale up the cluster

In [23]:
cluster.scale(jobs=2)

In [24]:
!squeue -u $USER

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
             96175       CPU dask-wor  valerio  R       2:56      1 somacpu059
             96176       CPU dask-wor  valerio  R       2:56      1 somacpu060


In [25]:
client

<Client: 'tcp://10.102.0.62:41808' processes=2 threads=16, memory=18.62 GiB>

### From here everything is the same as with LocalCluster

In [26]:
import numpy, dask.array

def calculate_pi(size_in_bytes, number_of_chunks):
    
    """Calculate pi using a Monte Carlo method."""
    
    array_shape = (int(size_in_bytes / 8 / 2), 2)
    chunk_size = (int(array_shape[0] / number_of_chunks), 2)
    
    # 2D random positions array using dask.array
    xy = dask.array.random.uniform(
        low=0.0, high=1.0, size=array_shape,
        # specify chunk size, i.e. task number
        chunks=chunk_size )
  
    xy_inside_circle = (xy ** 2).sum(axis=1) < 1 # boolean

    pi = 4 * xy_inside_circle.sum() / xy_inside_circle.size
    
    # start Dask calculation
    pi = pi.compute()

    print(f"\nfrom {xy.nbytes / 1e9} GB randomly chosen positions")
    print(f"   pi estimate: {pi}")
    print(f"   pi error: {abs(pi - numpy.pi)}\n")
    # display(xy)
    
    return pi

### Let's calculate again...

In [27]:
%time pi = calculate_pi(size_in_bytes=10_000_000_000, number_of_chunks=100) # 10 GB


from 10.0 GB randomly chosen positions
   pi estimate: 3.1416472896
   pi error: 5.463601020672115e-05

CPU times: user 2.82 s, sys: 384 ms, total: 3.2 s
Wall time: 49.1 s


In [28]:
%time pi = calculate_pi(size_in_bytes=100_000_000_000, number_of_chunks=250) # 100 GB


from 100.0 GB randomly chosen positions
   pi estimate: 3.14160216256
   pi error: 9.508970206795198e-06

CPU times: user 1.87 s, sys: 161 ms, total: 2.04 s
Wall time: 19.9 s


In [14]:
%time pi = calculate_pi(size_in_bytes=1_000_000_000_000, number_of_chunks=2_000) # 1 TB


from 1000.0 GB randomly chosen positions
   pi estimate: 3.14159954144
   pi error: 6.88785020708238e-06

CPU times: user 4.97 s, sys: 431 ms, total: 5.4 s
Wall time: 46.5 s


### We can easily scale down the cluster

In [32]:
cluster.scale(jobs=2)

In [33]:
!squeue -u $USER

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
             96175       CPU dask-wor  valerio  R       6:22      1 somacpu059
             96176       CPU dask-wor  valerio  R       6:22      1 somacpu060


### And we can scale up the cluster whenever needed

In [17]:
cluster.scale(jobs=16)

In [18]:
!squeue -u $USER

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON) 
             55462   cluster dask-wor smomw122 CG       0:55      1 neshcl266 
             55463   cluster dask-wor smomw122 CG       0:55      1 neshcl100 
             55464   cluster dask-wor smomw122 CG       0:55      1 neshcl101 
             55468   cluster dask-wor smomw122 CG       0:55      1 neshcl253 
             55474   cluster dask-wor smomw122 PD       0:00      1 (Resources) 
             55475   cluster dask-wor smomw122 PD       0:00      1 (Priority) 
             55470   cluster dask-wor smomw122  R       0:18      1 neshcl213 
             55471   cluster dask-wor smomw122  R       0:18      1 neshcl306 
             55472   cluster dask-wor smomw122  R       0:18      1 neshcl322 
             55473   cluster dask-wor smomw122  R       0:18      1 neshcl323 
             55461   cluster dask-wor smomw122  R       0:55      1 neshcl251 
             55465   cluster dask-wor smom

In [19]:
client

Client Scheduler: tcp://172.18.4.100:33939 Dashboard: http://172.18.4.100:8787/status,Cluster Workers: 8 Cores: 64 Memory: 384.00 GB


### Let's calculate again...

In [26]:
# %time pi = calculate_pi(size_in_bytes=1_000_000_000_000, number_of_chunks=2_000) # 1 TB

In [21]:
# %time pi = calculate_pi(size_in_bytes=10_000_000_000_000, number_of_chunks=10_000) # 10 TB

### Note, we could also adaptively scale the jobqueue cluster!

Dask jobqueue is able to scale total worker number based on problem size. You can also specify a target duration.

In [22]:
cluster.adapt(
    minimum_jobs=2, maximum_jobs=16,
)

In [23]:
%time pi = calculate_pi(size_in_bytes=10_000_000_000, number_of_chunks=100) # 10 GB


from 10.0 GB randomly chosen positions
   pi estimate: 3.1415523008
   pi error: 4.0352789793196564e-05

CPU times: user 254 ms, sys: 7.25 ms, total: 261 ms
Wall time: 3.07 s


In [24]:
%time pi = calculate_pi(size_in_bytes=1_000_000_000_000, number_of_chunks=1_000) # 1 TB


from 1000.0 GB randomly chosen positions
   pi estimate: 3.141595556288
   pi error: 2.902698206685983e-06

CPU times: user 3.08 s, sys: 284 ms, total: 3.37 s
Wall time: 48 s


In [25]:
# %time pi = calculate_pi(size_in_bytes=10_000_000_000_000, number_of_chunks=10_000) # 10 TB